In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

In [3]:
train_df = pd.read_csv('/root/tempfile/train_final.csv')
test_df = pd.read_csv('/root/tempfile/test_final.csv')

In [6]:
train_df.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,authorized_flag_count,authorized_flag_mean,authorized_flag_sum,auth_city_id_nunique,...,final_unauth_log_amount_sum_mean,final_unauth_log_amount_sum_std,final_unauth_log_amount_mean_mean,final_unauth_log_amount_mean_std,final_unauth_log_amount_max_mean,final_unauth_log_amount_max_std,final_unauth_log_amount_min_mean,final_unauth_log_amount_min_std,final_unauth_log_amount_std_mean,final_unauth_log_amount_std_std
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283,283,0.954064,270,9,...,-65.574414,45.036044,-2.630992,0.384338,-1.043115,0.894565,-4.036957,0.421211,0.837064,0.207392
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913,356,0.969101,345,9,...,-67.151705,52.006621,-2.906739,0.592226,-0.540343,1.412495,-4.394119,0.341552,1.042343,0.331018
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056,44,0.954545,42,5,...,-9.346733,6.496794,-2.843535,0.430235,-2.391969,0.665803,-3.205255,0.408528,0.574998,0.402799
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495,84,1.000000,84,7,...,-31.449652,30.894868,-2.793715,0.593955,-1.058167,0.878666,-4.088342,0.596358,1.018080,0.264283
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749,169,0.970414,164,7,...,-66.775135,31.815563,-2.376776,0.282495,0.268834,1.155971,-4.325026,0.773617,1.099192,0.157305


In [7]:
Id = test_df.card_id.values

droplist = [i for i in train_df.columns if train_df[i].dtype == object]

train_df.drop(droplist, axis=1, inplace=True)
test_df.drop(droplist, axis=1, inplace=True)

In [8]:
y_train = train_df.target.values
train_df.drop('target', axis=1, inplace=True)


In [10]:

x_train = train_df.values.astype('float')
x_test = test_df.values.astype('float')

In [9]:
category_feature = [
    'feature_1', 
    'feature_2', 
    'feature_3',
    'auth_state_id_cal_mode', 
    'auth_city_id_cal_mode',
    'auth_merchant_category_id_cal_mode',
    'auth_subsector_id_cal_mode','auth_dayofweek_cal_mode']

In [14]:
lgb_model = lgb.LGBMRegressor(objective='regression',
                              num_leaves=50,
                              learning_rate=0.1, 
                              n_estimators=43, 
                              max_depth=8,
                              metric='rmse',
                              bagging_fraction = 0.8,
                              feature_fraction = 0.8)

params_test1={
    'max_depth': range(3,8,2),
    'num_leaves':range(50, 170, 30)
}
gsearch1 = GridSearchCV(estimator=lgb_model, 
                        param_grid=params_test1,
                        scoring='neg_mean_squared_error', 
                        cv=5,
                        verbose=1, 
                        n_jobs=3)

gs1 = gsearch1.fit(x_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:  3.4min
[Parallel(n_jobs=3)]: Done  60 out of  60 | elapsed:  5.1min finished


In [15]:
print(gs1.best_params_, gs1.best_score_)

{'max_depth': 5, 'num_leaves': 50} -13.77245892281742


In [ ]:
params_test2={
    'max_depth': [4,5,6],
    'num_leaves':range(10,60,10)
}

gsearch2= GridSearchCV(estimator=lgb_model, 
                        param_grid=params_test2,
                        scoring='neg_mean_squared_error', 
                        cv=5,
                        verbose=1, 
                        n_jobs=3)

gs2 = gsearch2.fit(x_train, y_train)

In [ ]:
print(gs2.best_params_, gs2.best_score_)

In [ ]:
params_test3= {
    'max_depth': [6],
    'num_leaves':range(16,24,2)
}

gsearch3= GridSearchCV(estimator=lgb_model, 
                        param_grid=params_test3,
                        scoring='neg_mean_squared_error', 
                        cv=5,
                        verbose=1, 
                        n_jobs=3)

gs3 = gsearch3.fit(x_train, y_train)

In [ ]:
print(gs3.best_params_, gs3.best_score_)

In [ ]:
gs3.estimator

In [ ]:
params_test4= {
    'max_depth': [6],
    'num_leaves':[18],
    'min_child_samples': [18],
    'min_child_weight':[0.001],
    'feature_fraction': [0.5, 0.6, 0.7, 0.8, 0.9],   
    'bagging_fraction': [0.6, 0.7, 0.8, 0.9, 1.0]

}

gsearch4= GridSearchCV(estimator=lgb_model, 
                        param_grid=params_test4,
                        scoring='neg_mean_squared_error', 
                        cv=5,
                        verbose=1, 
                        n_jobs=3)

gs4 = gsearch4.fit(x_train, y_train)

In [ ]:
print(gs4.best_params_, gs4.best_score_)

In [ ]:
params_test5= {
    'max_depth': [6],
    'num_leaves':[18],
    'min_child_samples': [18],
    'min_child_weight':[0.001],
    'feature_fraction': [0.8, 0.85, 0.9, 0.95],    
    'bagging_fraction':[0.4, 0.45, 0.5, 0.55, 0.6, 0.65]
   # 'reg_alpha': [0, 0.001, 0.01, 0.03, 0.08, 0.3, 0.5],  
    #'reg_lambda': [0, 0.001, 0.01, 0.03, 0.08, 0.3, 0.5]

}

gsearch5= GridSearchCV(estimator=lgb_model, 
                        param_grid=params_test5,
                        scoring='neg_mean_squared_error', 
                        cv=5,
                        verbose=1, 
                        n_jobs=3)

gs5 = gsearch5.fit(x_train, y_train)

In [ ]:
print(gs5.best_params_, gs5.best_score_)

In [ ]:
params_test6= {
    'max_depth': [6],
    'num_leaves':[18],
    'min_child_samples': [18],
    'min_child_weight':[0.001],
    'feature_fraction': [0.9],    
    'bagging_fraction':[0.4],
    'reg_alpha': [0, 0.001, 0.01, 0.03, 0.08, 0.3, 0.5],  
    'reg_lambda': [0, 0.001, 0.01, 0.03, 0.08, 0.3, 0.5]

}

gsearch6 = GridSearchCV(estimator=lgb_model, 
                        param_grid=params_test6,
                        scoring='neg_mean_squared_error', 
                        cv=5,
                        verbose=1, 
                        n_jobs=3)

gs6 = gsearch6.fit(x_train, y_train)

In [ ]:
print(gs6.best_params_, gs6.best_score_)

In [ ]:
params_test7= {
    'max_depth': [6],
    'num_leaves':[18],
    'min_child_samples': [18],
    'min_child_weight':[0.001],
    'feature_fraction': [0.9],    
    'bagging_fraction':[0.4],
    'reg_alpha': [0.5, 0.55, 0.6, 0.7, 0.8],  
    'reg_lambda': [0, 0.0001, 0.0005, 0.001, 0.0015, 0.002]

}

gsearch7 = GridSearchCV(estimator=lgb_model, 
                        param_grid=params_test7,
                        scoring='neg_mean_squared_error', 
                        cv=5,
                        verbose=1, 
                        n_jobs=3)

gs7 = gsearch7.fit(x_train, y_train)

In [ ]:
print(gs7.best_params_, gs7.best_score_)

In [ ]:
params_test7= {
    'max_depth': 6,
    'num_leaves':18,
    'min_child_samples': 18,
    'min_child_weight':0.001,
    'feature_fraction': 0.9,    
    'bagging_fraction':0.4,
    'reg_alpha': 0.5,  
    'reg_lambda': 0.002,
    'learning_rate': 0.005
}

data_train = lgb.Dataset(train_df, y_train, silent=True)
lgb_model = lgb.train(
    params_test7,
    data_train,
    num_boost_round=10000, 
    verbose_eval=100)


In [ ]:
y_test = lgb_model.predict(x_test)

In [ ]:
submission = pd.DataFrame({'card_id':Id, 'target':y_test})

In [ ]:
submission.target.hist(bins=100)

In [ ]:
plt.hist(y_train,bins=100)
plt.show()

In [ ]:
important = pd.Series(lgb_model.feature_importance(), index=featurename).sort_values(ascending=True)

In [ ]:
plt.figure(figsize=[10,40])
plt.barh(important.index, important)
plt.show()

In [ ]:
submission.to_csv('/home/leechh/tempfile/elo/submission120901.csv')